Standard form appears to be working. A dictionary object constructed without the low-rank approximations yields a nearly identical state after 1 iteration.

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import post_process_grad as ppg

fltrSz = (3,3)
fftSz = (8,8)
noc = 5
nof = 4
rho = 1.0
nots = 100
lraParam = {'n_components': 5}
dtype = tf.complex128

D = tf.random.normal(shape=(1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype)
dictObj = fctr.dictionary_object2D_init(fftSz=fftSz,D=D,rho=rho,name='dictObj1',lraParam=lraParam)
dictObj2 = fctr.dictionary_object2D_init_full(fftSz=fftSz,D=D,rho=rho,name='dictObj2',lraParam=lraParam)


w = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))

x = tf.linalg.matmul(B,w,adjoint_a=True)
xi = tf.keras.layers.Input(x.shape[1:],dtype=dtype)
y = rho*xi + tf.linalg.matmul(B,tf.linalg.matmul(B,xi),adjoint_a=True)
z = dictObj.qinv(y)
z2 = dictObj2.qinv(y)
modelA = tf.keras.Model(xi,z)
modelB = tf.keras.Model(xi,z2)
xt = x[slice(0,10,1),slice(None),slice(None),slice(None),slice(None)]

optimizerA=tf.keras.optimizers.SGD(0.001)
optimizerB=tf.keras.optimizers.SGD(0.001)
with tf.GradientTape() as tape:
    zt = modelA(xt)
    loss = tf.keras.losses.MSE(xt,zt)

grad = tape.gradient(target=loss,sources=modelA.trainable_variables)
optimizerA.apply_gradients(zip(grad,modelA.trainable_variables))

with tf.GradientTape() as tape2:
    zt2 = modelB(xt)
    loss2 = tf.keras.losses.MSE(xt,zt2)

grad2 = tape2.gradient(target=loss2,sources=modelB.trainable_variables)
optimizerB.apply_gradients(zip(grad2,modelB.trainable_variables))

print(tf.math.reduce_max(tf.abs(dictObj2.dhmul.Df - dictObj2.dhmul.Dfprev)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Df - dictObj.dhmul.Dfprev)))

a = dictObj._dict_update()
b = dictObj2._dict_update()
currMatL = tf.linalg.matmul(dictObj.qinv.L,dictObj.qinv.L,adjoint_b=True)
currMatL2 = tf.linalg.matmul(dictObj2.qinv.L,dictObj2.qinv.L,adjoint_b=True)

print(tf.math.reduce_max(tf.abs(currMatL - currMatL2)))
print(tf.math.reduce_max(tf.abs(dictObj.qinv.L - dictObj2.qinv.L)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Df - dictObj2.dhmul.Df)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Dfprev - dictObj2.dhmul.Dfprev)))
print(tf.math.reduce_max(tf.abs(dictObj.D - dictObj2.D)))
print(tf.math.reduce_max(tf.abs(dictObj.R - dictObj2.R)))


tf.Tensor(17169.65874886973, shape=(), dtype=float64)
tf.Tensor(17169.65874886973, shape=(), dtype=float64)
tf.Tensor(3.552713678800501e-14, shape=(), dtype=float64)
tf.Tensor(3.375077994860476e-14, shape=(), dtype=float64)
tf.Tensor(3.375077994860476e-14, shape=(), dtype=float64)
tf.Tensor(2.980948821118545e-14, shape=(), dtype=float64)
tf.Tensor(3.5083047578154947e-14, shape=(), dtype=float64)
tf.Tensor(3.061093045708674e-14, shape=(), dtype=float64)
tf.Tensor(3.375077994860476e-14, shape=(), dtype=float64)
tf.Tensor(3.6415315207705135e-14, shape=(), dtype=float64)
tf.Tensor(3.197442310920451e-14, shape=(), dtype=float64)
tf.Tensor(3.019806626980426e-14, shape=(), dtype=float64)
tf.Tensor(2.7000623958883807e-13, shape=(), dtype=float64)
tf.Tensor(4.773959005888173e-14, shape=(), dtype=float64)
tf.Tensor(1.4432899320127035e-14, shape=(), dtype=float64)
tf.Tensor(1.4432899320127035e-14, shape=(), dtype=float64)
tf.Tensor(4.9960036108132044e-15, shape=(), dtype=float64)
tf.Tensor(1.1102

Woodbury Form is broken for some reason.

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import post_process_grad as ppg

fltrSz = (3,3)
fftSz = (8,8)
noc = 3
nof = 8
rho = 1.0
nots = 100
lraParam = {'n_components': 3}
dtype = tf.complex128

D = tf.random.normal(shape=(1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype)
dictObj = fctr.dictionary_object2D_init(fftSz=fftSz,D=D,rho=rho,name='dictObj1',lraParam=lraParam)
dictObj2 = fctr.dictionary_object2D_init_full(fftSz=fftSz,D=D,rho=rho,name='dictObj2',lraParam=lraParam)


w = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))

x = tf.linalg.matmul(B,w,adjoint_a=True)
xi = tf.keras.layers.Input(x.shape[1:],dtype=dtype)
y = rho*xi + tf.linalg.matmul(B,tf.linalg.matmul(B,xi),adjoint_a=True)
z = dictObj.qinv(y)
z2 = dictObj2.qinv(y)
modelA = tf.keras.Model(xi,z)
modelB = tf.keras.Model(xi,z2)
xt = x[slice(0,10,1),slice(None),slice(None),slice(None),slice(None)]

with tf.GradientTape() as tape:
    zt = modelA(xt)
    loss = tf.keras.losses.MSE(xt,zt)

grad = tape.gradient(target=loss,sources=modelA.trainable_variables)

with tf.GradientTape() as tape2:
    zt2 = modelB(xt)
    loss2 = tf.keras.losses.MSE(xt,zt2)

grad2 = tape2.gradient(target=loss2,sources=modelB.trainable_variables)

optimizerA=tf.keras.optimizers.SGD(0.001)
optimizerB=tf.keras.optimizers.SGD(0.001)

optimizerA.apply_gradients(zip(grad,modelA.trainable_variables))

optimizerB.apply_gradients(zip(grad2,modelB.trainable_variables))


print(tf.math.reduce_max(tf.abs(dictObj2.dhmul.Df - dictObj2.dhmul.Dfprev)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Df - dictObj.dhmul.Dfprev)))
Lprev = tf.Variable(dictObj.qinv.L)
a=dictObj._dict_update()
b=dictObj2._dict_update()
currMatL = tf.linalg.matmul(dictObj.qinv.L,dictObj.qinv.L,adjoint_b=True)
currMatL2 = tf.linalg.matmul(dictObj2.qinv.L,dictObj2.qinv.L,adjoint_b=True)

print(tf.math.reduce_max(tf.abs(dictObj.qinv.L - Lprev)))
print(tf.math.reduce_max(tf.abs(currMatL - currMatL2)))
print(tf.math.reduce_max(tf.abs(dictObj.qinv.L - dictObj2.qinv.L)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Df - dictObj2.dhmul.Df)))
print(tf.math.reduce_max(tf.abs(dictObj.dhmul.Dfprev - dictObj2.dhmul.Dfprev)))
print(tf.math.reduce_max(tf.abs(dictObj.D - dictObj2.D)))
print(tf.math.reduce_max(tf.abs(dictObj.R - dictObj2.R)))

tf.Tensor(21362.531734252658, shape=(), dtype=float64)
tf.Tensor(21362.531734252658, shape=(), dtype=float64)
tf.Tensor(2.90878432451791e-14, shape=(), dtype=float64)
tf.Tensor(3.042011087472929e-14, shape=(), dtype=float64)
tf.Tensor(3.419486915845482e-14, shape=(), dtype=float64)
tf.Tensor(2.842170943040401e-14, shape=(), dtype=float64)
tf.Tensor(2.1892090361011114, shape=(), dtype=float64)
<tf.Variable 'UnreadVariable' shape=(1, 8, 5, 3, 3) dtype=complex128, numpy=
array([[[[[ 1.04764177e+01 +0.j        ,  8.01434703e+00 +0.j        ,
            1.51896342e+01 +0.j        ],
          [ 8.01434703e+00 +0.j        ,  5.14127980e+01 +0.j        ,
            4.73930059e+01 +0.j        ],
          [ 1.51896342e+01 +0.j        ,  4.73930059e+01 +0.j        ,
            6.13532790e+01 +0.j        ]],

         [[ 2.57127306e+01 +0.j        ,  1.03296899e+00 -2.54734388j,
            1.17663754e+01 -0.95295508j],
          [ 1.03296899e+00 +2.54734388j,  5.71035828e+01 +0.j        ,
  

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import post_process_grad as ppg
import util

fltrSz = (2,2)
fftSz = (3,3)
noc = 3
nof = 8
rho = 1.0
nots = 100
lraParam = {'n_components': 3}
dtype = tf.complex128

D = tf.random.normal(shape=(1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype)
dictObj = fctr.dictionary_object2D_init(fftSz=fftSz,D=D,rho=rho,name='dictObj1',lraParam=lraParam)
dictObj2 = fctr.dictionary_object2D_init_full(fftSz=fftSz,D=D,rho=rho,name='dictObj2',lraParam=lraParam)


w = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,2,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))

x = tf.linalg.matmul(B,w,adjoint_a=True)
xi = tf.keras.layers.Input(x.shape[1:],dtype=dtype)
y = rho*xi + tf.linalg.matmul(B,tf.linalg.matmul(B,xi),adjoint_a=True)
z = dictObj.qinv(y)
z2 = dictObj2.qinv(y)
modelA = tf.keras.Model(xi,z)
modelB = tf.keras.Model(xi,z2)
xt = x[slice(0,10,1),slice(None),slice(None),slice(None),slice(None)]

optimizerA=tf.keras.optimizers.SGD(0.001)
optimizerB=tf.keras.optimizers.SGD(0.001)
with tf.GradientTape() as tape:
    zt = modelA(xt)
    loss = tf.keras.losses.MSE(xt,zt)

grad = tape.gradient(target=loss,sources=modelA.trainable_variables)
optimizerA.apply_gradients(zip(grad,modelA.trainable_variables))

with tf.GradientTape() as tape2:
    zt2 = modelB(xt)
    loss2 = tf.keras.losses.MSE(xt,zt2)

grad2 = tape2.gradient(target=loss2,sources=modelB.trainable_variables)
optimizerB.apply_gradients(zip(grad2,modelB.trainable_variables))

dictObj2._dict_update()
L = tf.Variable(dictObj.qinv.L)
currMatL = tf.Variable(tf.linalg.matmul(L,L,adjoint_b=True))

Dnew = dictObj.get_constrained_D(dictObj.dhmul.Df)

# compute low rank approximation of the update
theUpdate = Dnew - dictObj.D
U,V,approx = fctr.stack_svd(theUpdate,5,**dictObj.lraParam)

# Compute DFT (The conjugate is necessary because F{A} = F{U}F{V}^T
Uf = util.complexNum(U)
Vf = tf.math.conj(transf.fft2d_inner(dictObj.fftSz)(V))

theUpdatef = transf.fft2d_inner(dictObj.fftSz)(theUpdate)
Uf2,Vf2,approxf2 = fctr.stack_svd(theUpdatef,5,**dictObj.lraParam)

print(tf.math.reduce_max(tf.math.abs(approxf2 - transf.fft2d_inner(dictObj.fftSz)(approx))))
print(tf.math.reduce_max(tf.math.abs(theUpdatef - tf.linalg.matmul(Uf,Vf,adjoint_b=True))))

rank1update2 = tf.linalg.matmul(tf.linalg.matmul(Uf2,Vf2,adjoint_b = True),tf.linalg.matmul(Uf2,Vf2,adjoint_b = True),adjoint_b = True)

# Update Decomposition and Frequency-Domain Dictionary
Lr1 = tf.Variable(dictObj._rank1_updates(Uf,Vf,L))
VhV = tf.math.reduce_sum(Vf2*tf.math.conj(Vf2),axis=3,keepdims=True)
rank1update = tf.linalg.matmul(VhV*Uf2,Uf2,adjoint_b=True)

currMatLr1u = tf.Variable(tf.linalg.matmul(dictObj.qinv.L,dictObj.qinv.L,adjoint_b=True))

print(tf.math.reduce_sum(tf.math.abs(rank1update2 - rank1update)))


Dv = tf.linalg.matmul(dictObj.dhmul.Dfprev,Vf)
rank2update = tf.linalg.matmul(Dv,Uf,adjoint_b=True) + tf.linalg.matmul(Uf,Dv,adjoint_b=True)

eigvals,eigvecs,asVec = dictObj._get_eigen_decomp(Uf,Vf,dictObj.dhmul.Dfprev)
eigsum = 0.
for vals,vecs in zip(eigvals,eigvecs):
    for val,vec in zip(tf.unstack(vals,axis=0),tf.unstack(vecs,axis=0)):
        eigsum = eigsum + util.addDim(util.addDim(val))*tf.linalg.matmul(util.addDim(vec),util.addDim(vec),adjoint_b = True)

#print(tf.reduce_max(tf.math.abs(rank2update - eigsum)))

mat2 = tf.linalg.matmul(dictObj2.qinv.L,dictObj2.qinv.L,adjoint_b=True)

#print(tf.reduce_max(tf.math.abs(mat2 - currMatL - rank1update - rank2update)))
#Lr2 = dictObj._eig_chol_update(eigvals,eigvecs,dictObj.qinv.L)
#currMatLr2u = tf.Variable(tf.linalg.matmul(dictObj.qinv.L,dictObj.qinv.L,adjoint_b = True))


#print(tf.reduce_max(tf.math.abs(currMatLr2u - currMatLr1u - eigsum)))

#print(currMatLr1u)

#Lr2,asVec = dictObj._rank2_updates(Uf,Vf,dictObj.dhmul.Dfprev,Lr1)
#Lr2 = tf.Variable(Lr2)

#currMatLr2u = tf.linalg.matmul(Lr2,Lr2,adjoint_b=True)

#print(rank2update)
#print(currMatLr2u - currMatLr1u - rank2update)

tf.Tensor(5.9580819677934555e-15, shape=(), dtype=float64)
tf.Tensor(3.4684476073050936e-15, shape=(), dtype=float64)
tf.Tensor(454.8322177299515, shape=(), dtype=float64)


In [2]:
currMatLr2u = tf.linalg.matmul(Lr2,Lr2,adjoint_b=True)
print(currMatLr2u - currMatLr1u)

tf.Tensor(
[[[[[ 0.00000000e+00+0.00000000e+00j -1.57859836e-16+5.47382213e-48j]
    [-1.57859836e-16-5.47382213e-48j  4.44089210e-16+0.00000000e+00j]]

   [[-1.27506745e+01+0.00000000e+00j -1.39378880e+00+3.48980075e-01j]
    [-1.39378880e+00-3.48980075e-01j -1.35246946e+00+0.00000000e+00j]]]


  [[[-1.62416165e+01+0.00000000e+00j -1.16452892e+01-1.51328808e-01j]
    [-1.16452892e+01+1.51328808e-01j -1.79914575e+01+0.00000000e+00j]]

   [[-2.18473207e+01+0.00000000e+00j -9.24532987e-01+7.49635854e+00j]
    [-9.24532987e-01-7.49635854e+00j  7.14730078e+00+0.00000000e+00j]]]


  [[[-1.62416165e+01+0.00000000e+00j -1.16452892e+01+1.51328808e-01j]
    [-1.16452892e+01-1.51328808e-01j -1.79914575e+01+0.00000000e+00j]]

   [[-1.19846682e+01+0.00000000e+00j -6.01085037e+00-2.19029558e-01j]
    [-6.01085037e+00+2.19029558e-01j -1.28612376e+01+0.00000000e+00j]]]]], shape=(1, 3, 2, 2, 2), dtype=complex128)


In [3]:
print(eigvecs[1][1])

tf.Tensor(
[[[[ 3.66808981e-14+1.82620900e-14j  1.72994584e-14+8.61277347e-15j]
   [ 3.28235444e-02+3.30082630e-02j -9.96923833e-01+6.30550681e-02j]]

  [[ 8.09338645e-01-9.47784763e-03j -5.86971813e-01+1.85800561e-02j]
   [ 4.23628772e-01-2.83566961e-01j -7.76127389e-01-3.71153229e-01j]]

  [[ 8.09338645e-01+9.47784763e-03j -5.86971813e-01-1.85800561e-02j]
   [ 2.06275149e-01+3.73099411e-01j -8.97641899e-01+1.11742621e-01j]]]], shape=(1, 3, 2, 2), dtype=complex128)


In [4]:
VhV

<tf.Tensor: shape=(1, 8, 5, 1, 3), dtype=complex128, numpy=
array([[[[[1.24372066+0.j, 0.65238577+0.j, 0.12519912+0.j]],

         [[1.29827414+0.j, 0.83938382+0.j, 0.70425554+0.j]],

         [[1.24746564+0.j, 1.03418915+0.j, 1.46738332+0.j]],

         [[0.93854587+0.j, 0.86603954+0.j, 1.33271449+0.j]],

         [[0.73498807+0.j, 0.69008266+0.j, 1.01397429+0.j]]],


        [[[1.08630945+0.j, 0.77098751+0.j, 0.58690447+0.j]],

         [[1.25054204+0.j, 0.9121986 +0.j, 0.43655462+0.j]],

         [[1.01606036+0.j, 0.98475371+0.j, 0.71677172+0.j]],

         [[0.70281923+0.j, 0.79601501+0.j, 0.76543103+0.j]],

         [[0.99947963+0.j, 0.76440402+0.j, 0.79984342+0.j]]],


        [[[0.79667395+0.j, 0.88456978+0.j, 1.18298464+0.j]],

         [[1.14937826+0.j, 0.96023416+0.j, 0.80710283+0.j]],

         [[1.07615086+0.j, 1.26854068+0.j, 0.60975228+0.j]],

         [[0.91165808+0.j, 1.23011594+0.j, 0.61948086+0.j]],

         [[1.25864392+0.j, 0.90349772+0.j, 0.69258987+0.j]]],


    